In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
import csv
# from api_key import api_key
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [2]:
export_url = (f"https://api.census.gov/data/timeseries/intltrade/exports")
import_url = (f"https://api.census.gov/data/timeseries/intltrade/imports")
china_code = "5700"
usa_code = "1000"
month = []
year = []

In [3]:
export_by_commodity = requests.get(f"{export_url}/hs?get=E_COMMODITY,E_COMMODITY_SDESC,ALL_VAL_MO,ALL_VAL_YR&time=from+2015-01&COMM_LVL=HS2&CTY_CODE={china_code}").json()


In [4]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4745:
    count += 1
    movalue.append(export_by_commodity[count][2])
    yrvalue.append(export_by_commodity[count][3])
    desc.append(export_by_commodity[count][1])
    period.append(export_by_commodity[count][4])
    code.append(export_by_commodity[count][0])
china_data_export = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [5]:
import_by_commodity = requests.get(f"{import_url}/hs?get=I_COMMODITY,I_COMMODITY_SDESC,GEN_VAL_MO,GEN_VAL_YR&time=from+2015-01&COMM_LVL=HS2&CTY_CODE={china_code}").json()


In [6]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4802:
    count += 1
    movalue.append(import_by_commodity[count][2])
    yrvalue.append(import_by_commodity[count][3])
    desc.append(import_by_commodity[count][1])
    period.append(import_by_commodity[count][4])
    code.append(import_by_commodity[count][0])
china_data_import = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [7]:
china_data_export["YTD Value"] =pd.to_numeric(china_data_export["YTD Value"])
china_data_import["YTD Value"] =pd.to_numeric(china_data_import["YTD Value"])

In [10]:
def yearly_data(trade,year):
    data = trade[trade["Period"].str.contains(year)]
    data = data.groupby(["Comm Code","Commodity"])["YTD Value"].sum()
    data = pd.DataFrame({"total" : data})
    data = data.nlargest(10,"total")
    data = data.reset_index()
    
    return data
    
    

In [8]:
# first_2015 = china_data_export[china_data_export["Period"].str.contains("2015")]
china_data_import["YTD Value"].sum()

12491142219444

In [9]:
# data_2015 = first_2015.groupby(["Comm Code","Commodity"])["YTD Value"].sum()
china_data_export["YTD Value"].sum()

3041011721625

In [ ]:
# test= pd.DataFrame({"total" : data_2015})

In [11]:
data_2015_import = yearly_data(china_data_import,"2015")
data_2015_export = yearly_data(china_data_export,"2015")

data_2016_import = yearly_data(china_data_import,"2016")
data_2016_export = yearly_data(china_data_export,"2016")

data_2017_import = yearly_data(china_data_import,"2017")
data_2017_export = yearly_data(china_data_export,"2017")

data_2018_import = yearly_data(china_data_import,"2018")
data_2018_export = yearly_data(china_data_export,"2018")


In [15]:
# data_2015= test.nlargest(10,"total")
data_2015_import = data_2015_import.to_csv("data_2015_import.csv", index=False)
data_2015_export = data_2015_export.to_csv("data_2015_export.csv", index=False)

data_2016_import = data_2016_import.to_csv("data_2016_import.csv", index=False)
data_2016_export = data_2016_export.to_csv("data_2016_export.csv", index=False)

data_2017_import = data_2017_import.to_csv("data_2017_import.csv", index=False)
data_2017_export = data_2017_export.to_csv("data_2017_export.csv", index=False)

data_2018_import = data_2018_import.to_csv("data_2018_import.csv", index=False)
data_2018_export = data_2018_export.to_csv("data_2018_export.csv", index=False)

In [14]:
data_2017_export

,Comm Code,Commodity,total
0,88,"AIRCRAFT, SPACECRAFT, AND PARTS THEREOF",89147329660
1,87,"VEHICLES, EXCEPT RAILWAY OR TRAMWAY, AND PARTS...",82677835172
2,84,"NUCLEAR REACTORS, BOILERS, MACHINERY ETC.; PARTS",82173538464
3,85,ELECTRIC MACHINERY ETC; SOUND,77689686416
4,12,"OIL SEEDS ETC.; MISC GRAIN, SEED, FRUIT, PLANT...",68518722389
5,90,"OPTIC, PHOTO ETC, MEDIC OR S",54679120250
6,27,"MINERAL FUEL, OIL ETC.; BITUMIN SUBST; MINERAL...",49165665941
7,39,PLASTICS AND ARTICLES THEREOF,37858863659
8,47,WOOD PULP ETC; RECOVD (WASTE & SCRAP) PPR & PPRBD,21983772355
9,44,WOOD AND ARTICLES OF WOOD; WOOD CHARCOAL,20120942427


In [ ]:
# data_2015 = data_2015.to_csv("data_2015.csv", index=False)